In [1]:
import pandas as pd
from openai import OpenAI
import random
import os
import re

In [635]:
category = "letterale"
idiom = "mandare a casa"

model ="gpt-4-turbo"

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key="OpenAI Key",
)

In [636]:
if not os.path.exists('response_data.csv'):
    pd.DataFrame(columns=["raw_response","category", "idiom", "type"]).to_csv('response_data.csv', index=False)


In [637]:
df_structured = pd.DataFrame(columns=["submission", "category", "idiom", "type"])

In [638]:
class ChatSession:
    def __init__(self, category, model=model):
        self.model = model
        system_message = f"Poiché l'italiano è la tua lingua madre, dovresti utilizzare il nostro linguaggio in modi ricchi e creativi. Per favore, evita di utilizzare strutture simili nelle tue frasi. Non dovresti costruire frasi usando nomi di persone! Devi usare le parole specificate nella richiesta in un modo reale, concreto o fisico. Non devi usare le parole specificate in senso figurato!"
        print(system_message)
        self.messages = [
            {"role": "system", "content": system_message}
        ]

    def get_completion(self, user_prompt):
        # Add the new user message
        self.messages.append({"role": "user", "content": user_prompt})
        
        # Make the API call
        response = client.chat.completions.create(
            model=self.model,
            messages=self.messages,
        )

        # Retrieve the response and append to messages to maintain context
        ai_response = response.choices[0].message.content.strip()
        self.messages.append({"role": "assistant", "content": ai_response})
        response_df = pd.DataFrame(columns=["raw_response", "idiom","type","category"])
        new_entry = {
            "raw_response": ai_response,
            "idiom": idiom,
            "type": type,
            "category":category
        }
        response_df = response_df.append(new_entry, ignore_index=True)
        response_df.to_csv('response_data.csv', mode='a', header=False, index=False)
        return ai_response

In [639]:
chat_session = ChatSession(category=category)

Poiché l'italiano è la tua lingua madre, dovresti utilizzare il nostro linguaggio in modi ricchi e creativi. Per favore, evita di utilizzare strutture simili nelle tue frasi. Non dovresti costruire frasi usando nomi di persone! Devi usare le parole specificate nella richiesta in un modo reale, concreto o fisico. Non devi usare le parole specificate in senso figurato!


In [650]:
type = "Zero-shot"
step = 11

In [641]:
idiom.split(" ")
sentences = ""
sonx = idiom.split(" ")[-1]
firstx = idiom.split(" ")[0]
for l in idiom.split(" "):
    if sonx == l:
        sentences = sentences +" e "+'"'+ l.lower()+ '"'
    elif firstx == l:
        sentences = sentences +" "+'"'+ l.lower() + '"'
    else:
        sentences = sentences +", "+'"'+ l.lower() + '"'

In [642]:
temp = sentences.strip()
prompt = f"""
Scrivi 5 esempi di frasi in cui {temp} sono usate per esprimere un'azione fisica. Puoi coniugare le parole specificate senza cambiarne le radici come preferisci. Puoi posizionare le parole specificate dove vuoi nella frase, ma devi costruire una frase che includa tutte le parole indicate.Per favore, evita di ripetere le stesse frasi. Restituisci solo le frasi come risposta. Non dovresti costruire frasi usando nomi di persone!
"""

In [643]:
print(prompt)


Scrivi 5 esempi di frasi in cui "mandare", "a" e "casa" sono usate per esprimere un'azione fisica. Puoi coniugare le parole specificate senza cambiarne le radici come preferisci. Puoi posizionare le parole specificate dove vuoi nella frase, ma devi costruire una frase che includa tutte le parole indicate.Per favore, evita di ripetere le stesse frasi. Restituisci solo le frasi come risposta. Non dovresti costruire frasi usando nomi di persone!



In [651]:
responses = []
for i in range(step):
    result = chat_session.get_completion(prompt)
    responses.append(result)

In [652]:
responses

["1. L'autista ha mandato le consegne rapidamente a casa.\n2. Ho mandato il mio computer portatile a casa perché l'ufficio era in ristrutturazione.\n3. Manda queste piante a casa in modo che possano essere piantate in giardino.\n4. L'ufficio postale ha mandato il pacco errato a casa mia.\n5. Il negozio di riparazioni manda gli oggetti riparati a casa del cliente.",
 '1. In negozio, decidono di mandare gli avanzi a casa dei dipendenti.\n2. Qui mandano sempre le fatture direttamente a casa dei clienti.\n3. Durante il trasloco, hanno mandato le scatole più grandi a casa per prima.\n4. Hai mai mandato un messaggio a casa per avvisare di un ritardo?\n5. Quando puoi, manda queste brochure a casa nostra per la distribuzione.',
 '1. Durante la pausa estiva, la scuola mandava i libri a casa degli studenti.\n2. La ditta di pulizie aveva mandato i furgoni a casa per rifornirsi di materiali.\n3. La biblioteca comunale manda i libri a casa dei lettori anziani.\n4. Il ristorante ha deciso di mandare

In [653]:
def parse_and_structure_data(raw_responses_list, idiom, category, response_type):
    # Initialize the list to store structured data
    structured_data = []
    # Iterate through each block of responses
    for block in raw_responses_list:
        # Split each block into individual sentences
        sentences = block.split('\n')
        # Remove the numbering from each sentence and strip leading/trailing whitespace
        clean_sentences = [sentence.split('. ', 1)[-1].strip() for sentence in sentences]
        # For each sentence, append a tuple with the structured data
        for sentence in clean_sentences:
            structured_data.append((sentence, category, idiom, response_type))
    return structured_data

In [654]:
structured_data = parse_and_structure_data(responses, idiom, category, type)

new_df= pd.DataFrame(structured_data, columns=["submission", "category", "idiom", "type"])
new_df

,submission,category,idiom,type
0,L'autista ha mandato le consegne rapidamente a...,letterale,mandare a casa,Zero-shot
1,Ho mandato il mio computer portatile a casa pe...,letterale,mandare a casa,Zero-shot
2,Manda queste piante a casa in modo che possano...,letterale,mandare a casa,Zero-shot
3,L'ufficio postale ha mandato il pacco errato a...,letterale,mandare a casa,Zero-shot
4,Il negozio di riparazioni manda gli oggetti ri...,letterale,mandare a casa,Zero-shot
5,"In negozio, decidono di mandare gli avanzi a c...",letterale,mandare a casa,Zero-shot
6,Qui mandano sempre le fatture direttamente a c...,letterale,mandare a casa,Zero-shot
7,"Durante il trasloco, hanno mandato le scatole ...",letterale,mandare a casa,Zero-shot
8,Hai mai mandato un messaggio a casa per avvisa...,letterale,mandare a casa,Zero-shot
9,"Quando puoi, manda queste brochure a casa nost...",letterale,mandare a casa,Zero-shot


In [655]:
new_df = new_df[new_df['submission']!=""]
new_df.reset_index(drop=True,inplace=True)
new_df


,submission,category,idiom,type
0,L'autista ha mandato le consegne rapidamente a...,letterale,mandare a casa,Zero-shot
1,Ho mandato il mio computer portatile a casa pe...,letterale,mandare a casa,Zero-shot
2,Manda queste piante a casa in modo che possano...,letterale,mandare a casa,Zero-shot
3,L'ufficio postale ha mandato il pacco errato a...,letterale,mandare a casa,Zero-shot
4,Il negozio di riparazioni manda gli oggetti ri...,letterale,mandare a casa,Zero-shot
5,"In negozio, decidono di mandare gli avanzi a c...",letterale,mandare a casa,Zero-shot
6,Qui mandano sempre le fatture direttamente a c...,letterale,mandare a casa,Zero-shot
7,"Durante il trasloco, hanno mandato le scatole ...",letterale,mandare a casa,Zero-shot
8,Hai mai mandato un messaggio a casa per avvisa...,letterale,mandare a casa,Zero-shot
9,"Quando puoi, manda queste brochure a casa nost...",letterale,mandare a casa,Zero-shot


In [656]:
df_structured = pd.concat([df_structured, new_df], ignore_index=True)
df_structured

,submission,category,idiom,type
0,"Manda queste scatole a casa subito, per favore.",letterale,mandare a casa,Zero-shot
1,"Ogni venerdì, il manager manda gli impiegati a...",letterale,mandare a casa,Zero-shot
2,"Durante l'allerta meteo, hanno mandato tutti g...",letterale,mandare a casa,Zero-shot
3,Potete mandare quel pacco grande al mio indiri...,letterale,mandare a casa,Zero-shot
4,Chi ha mandato queste lettere alla nostra casa?,letterale,mandare a casa,Zero-shot
5,L'autista ha mandato le consegne rapidamente a...,letterale,mandare a casa,Zero-shot
6,Ho mandato il mio computer portatile a casa pe...,letterale,mandare a casa,Zero-shot
7,Manda queste piante a casa in modo che possano...,letterale,mandare a casa,Zero-shot
8,L'ufficio postale ha mandato il pacco errato a...,letterale,mandare a casa,Zero-shot
9,Il negozio di riparazioni manda gli oggetti ri...,letterale,mandare a casa,Zero-shot


In [657]:
class ChatSession:
    def __init__(self, category, model=model):
        self.model = model
        system_message = f"Poiché l'italiano è la tua lingua madre, dovresti utilizzare il nostro linguaggio in modi ricchi e creativi. Per favore, evita di utilizzare strutture simili nelle tue frasi. Non dovresti costruire frasi usando nomi di persone! Devi usare le parole specificate nella richiesta in un modo reale, concreto o fisico. Non devi usare le parole specificate in senso figurato!"
        print(system_message)
        self.messages = [
            {"role": "system", "content": system_message}
        ]

    def get_completion(self, user_prompt):
        # Add the new user message
        self.messages.append({"role": "user", "content": user_prompt})
        
        # Make the API call
        response = client.chat.completions.create(
            model=self.model,
            messages=self.messages,
        )

        # Retrieve the response and append to messages to maintain context
        ai_response = response.choices[0].message.content.strip()
        self.messages.append({"role": "assistant", "content": ai_response})
        response_df = pd.DataFrame(columns=["raw_response", "idiom","type","category"])
        new_entry = {
            "raw_response": ai_response,
            "idiom": idiom,
            "type": type,
            "category":category
        }
        response_df = response_df.append(new_entry, ignore_index=True)
        response_df.to_csv('response_data.csv', mode='a', header=False, index=False)
        return ai_response

In [658]:
chat_session = ChatSession(category=category)

Poiché l'italiano è la tua lingua madre, dovresti utilizzare il nostro linguaggio in modi ricchi e creativi. Per favore, evita di utilizzare strutture simili nelle tue frasi. Non dovresti costruire frasi usando nomi di persone! Devi usare le parole specificate nella richiesta in un modo reale, concreto o fisico. Non devi usare le parole specificate in senso figurato!


In [669]:
type = "Enhanced-prompting"
step = 3

In [661]:
idiom.split(" ")
sentences = ""
sonx = idiom.split(" ")[-1]
firstx = idiom.split(" ")[0]
for l in idiom.split(" "):
    if sonx == l:
        sentences = sentences +" e "+'"'+ l.lower()+ '"'
    elif firstx == l:
        sentences = sentences +" "+'"'+ l.lower() + '"'
    else:
        sentences = sentences +", "+'"'+ l.lower() + '"'

In [662]:
temp = sentences.strip()
prompt = f"""
Scrivi 5 frasi come se fossero pronunciate da una persona, in cui {temp} sono utilizzate per descrivere un'azione fisica. Puoi coniugare le parole specificate nell'istruzione senza modificarne le radici come preferisci. Puoi inserire le parole specificate dove desideri nella frase, ma devi costruire una frase che includa tutte le parole indicate. Per favore, evita di ripetere le stesse frasi. Restituisci solo le frasi come risposta.
"""

In [663]:
print(prompt)


Scrivi 5 frasi come se fossero pronunciate da una persona, in cui "mandare", "a" e "casa" sono utilizzate per descrivere un'azione fisica. Puoi coniugare le parole specificate nell'istruzione senza modificarne le radici come preferisci. Puoi inserire le parole specificate dove desideri nella frase, ma devi costruire una frase che includa tutte le parole indicate. Per favore, evita di ripetere le stesse frasi. Restituisci solo le frasi come risposta.



In [670]:
responses = []
for i in range(step):
    result = chat_session.get_completion(prompt)
    responses.append(result)

In [671]:
responses

['1. Domani manderò a casa tua il divano che hai comprato.\n2. Ho mandato a casa mia le sedie nuove per la sala da pranzo.\n3. Questo weekend manderò a casa dei miei genitori le vecchie foto che ho trovato.\n4. Si è fatto tardi, mando i ragazzi a casa prima che piova.\n5. Dopo la festa, ho mandato tutti a casa con un pacchetto di dolci.',
 '1. Ho mandato a casa tua le chiavi che avevi perso.\n2. Manderò a casa gli utensili da giardino che mi hai prestato.\n3. Abbiamo mandato a casa il personale non essenziale a causa del maltempo.\n4. Mandai a casa le tortine avanzate dalla cena di ieri.\n5. Ogni volta che i giocattoli finiscono fuori posto, li mando subito a casa loro nello scatolone in garage.',
 '1. Stamattina ho mandato a casa i vestiti puliti che avevi lasciato qui.\n2. Dopo la riunione, manderò a casa tutti i partecipanti con le informazioni aggiornate.\n3. Quando visita la nonna, le mando sempre a casa delle verdure fresche dal mio giardino.\n4. Mando a casa il personale extra n

In [672]:
structured_data = parse_and_structure_data(responses, idiom, category, type)

new_df= pd.DataFrame(structured_data, columns=["submission", "category", "idiom", "type"])
new_df

,submission,category,idiom,type
0,Domani manderò a casa tua il divano che hai co...,letterale,mandare a casa,Enhanced-prompting
1,Ho mandato a casa mia le sedie nuove per la sa...,letterale,mandare a casa,Enhanced-prompting
2,Questo weekend manderò a casa dei miei genitor...,letterale,mandare a casa,Enhanced-prompting
3,"Si è fatto tardi, mando i ragazzi a casa prima...",letterale,mandare a casa,Enhanced-prompting
4,"Dopo la festa, ho mandato tutti a casa con un ...",letterale,mandare a casa,Enhanced-prompting
5,Ho mandato a casa tua le chiavi che avevi perso.,letterale,mandare a casa,Enhanced-prompting
6,Manderò a casa gli utensili da giardino che mi...,letterale,mandare a casa,Enhanced-prompting
7,Abbiamo mandato a casa il personale non essenz...,letterale,mandare a casa,Enhanced-prompting
8,Mandai a casa le tortine avanzate dalla cena d...,letterale,mandare a casa,Enhanced-prompting
9,Ogni volta che i giocattoli finiscono fuori po...,letterale,mandare a casa,Enhanced-prompting


In [673]:
new_df = new_df[new_df['submission']!=""]
new_df.reset_index(drop=True,inplace=True)
new_df


,submission,category,idiom,type
0,Domani manderò a casa tua il divano che hai co...,letterale,mandare a casa,Enhanced-prompting
1,Ho mandato a casa mia le sedie nuove per la sa...,letterale,mandare a casa,Enhanced-prompting
2,Questo weekend manderò a casa dei miei genitor...,letterale,mandare a casa,Enhanced-prompting
3,"Si è fatto tardi, mando i ragazzi a casa prima...",letterale,mandare a casa,Enhanced-prompting
4,"Dopo la festa, ho mandato tutti a casa con un ...",letterale,mandare a casa,Enhanced-prompting
5,Ho mandato a casa tua le chiavi che avevi perso.,letterale,mandare a casa,Enhanced-prompting
6,Manderò a casa gli utensili da giardino che mi...,letterale,mandare a casa,Enhanced-prompting
7,Abbiamo mandato a casa il personale non essenz...,letterale,mandare a casa,Enhanced-prompting
8,Mandai a casa le tortine avanzate dalla cena d...,letterale,mandare a casa,Enhanced-prompting
9,Ogni volta che i giocattoli finiscono fuori po...,letterale,mandare a casa,Enhanced-prompting


In [674]:
df_structured = pd.concat([df_structured, new_df], ignore_index=True)
df_structured

,submission,category,idiom,type
0,"Manda queste scatole a casa subito, per favore.",letterale,mandare a casa,Zero-shot
1,"Ogni venerdì, il manager manda gli impiegati a...",letterale,mandare a casa,Zero-shot
2,"Durante l'allerta meteo, hanno mandato tutti g...",letterale,mandare a casa,Zero-shot
3,Potete mandare quel pacco grande al mio indiri...,letterale,mandare a casa,Zero-shot
4,Chi ha mandato queste lettere alla nostra casa?,letterale,mandare a casa,Zero-shot
...,...,...,...,...
75,Stamattina ho mandato a casa i vestiti puliti ...,letterale,mandare a casa,Enhanced-prompting
76,"Dopo la riunione, manderò a casa tutti i parte...",letterale,mandare a casa,Enhanced-prompting
77,"Quando visita la nonna, le mando sempre a casa...",letterale,mandare a casa,Enhanced-prompting
78,Mando a casa il personale extra non appena fin...,letterale,mandare a casa,Enhanced-prompting


In [675]:
class ChatSession:
    def __init__(self, category, model=model):
        self.model = model
        system_message = f"Poiché l'italiano è la tua lingua madre, dovresti utilizzare il nostro linguaggio in modi ricchi e creativi. Per favore, evita di utilizzare strutture simili nelle tue frasi. Non dovresti costruire frasi usando nomi di persone! Devi usare le parole specificate nella richiesta in un modo reale, concreto o fisico. Non devi usare le parole specificate in senso figurato!"
        print(system_message)
        self.messages = [
            {"role": "system", "content": system_message}
        ]

    def get_completion(self, user_prompt):
        # Add the new user message
        self.messages.append({"role": "user", "content": user_prompt})
        
        # Make the API call
        response = client.chat.completions.create(
            model=self.model,
            messages=self.messages,
        )

        # Retrieve the response and append to messages to maintain context
        ai_response = response.choices[0].message.content.strip()
        self.messages.append({"role": "assistant", "content": ai_response})
        response_df = pd.DataFrame(columns=["raw_response", "idiom","type","category"])
        new_entry = {
            "raw_response": ai_response,
            "idiom": idiom,
            "type": type,
            "category":category
        }
        response_df = response_df.append(new_entry, ignore_index=True)
        response_df.to_csv('response_data.csv', mode='a', header=False, index=False)
        return ai_response

In [676]:
chat_session = ChatSession(category=category)

Poiché l'italiano è la tua lingua madre, dovresti utilizzare il nostro linguaggio in modi ricchi e creativi. Per favore, evita di utilizzare strutture simili nelle tue frasi. Non dovresti costruire frasi usando nomi di persone! Devi usare le parole specificate nella richiesta in un modo reale, concreto o fisico. Non devi usare le parole specificate in senso figurato!


In [686]:
type = "Enhanced-prompting"
step = 3

In [678]:
idiom.split(" ")
sentences = ""
sonx = idiom.split(" ")[-1]
firstx = idiom.split(" ")[0]
for l in idiom.split(" "):
    if sonx == l:
        sentences = sentences +" e "+'"'+ l.lower()+ '"'
    elif firstx == l:
        sentences = sentences +" "+'"'+ l.lower() + '"'
    else:
        sentences = sentences +", "+'"'+ l.lower() + '"'

In [679]:
temp = sentences.strip()
prompt = f"""

"""Crea 5 frasi con strutture diverse, in cui {temp} sono utilizzate per descrivere un'azione fisica. Puoi posizionare le parole specificate dove preferisci nella frase, ma devi costruire una frase che includa tutte le parole indicate. Per favore, evita di ripetere le stesse frasi. Restituisci solo le frasi come risposta.

In [680]:
print(prompt)


Crea 5 frasi con strutture diverse, in cui "mandare", "a" e "casa" sono utilizzate per descrivere un'azione fisica. Puoi posizionare le parole specificate dove preferisci nella frase, ma devi costruire una frase che includa tutte le parole indicate. Per favore, evita di ripetere le stesse frasi. Restituisci solo le frasi come risposta.



In [687]:
responses = []
for i in range(step):
    result = chat_session.get_completion(prompt)
    responses.append(result)

In [688]:
responses

["1. I vigili del fuoco hanno dovuto mandare tutti gli abitanti del palazzo a casa dei vicini a causa di un allarme gas.\n2. Il forte temporale ha spinto l'insegnante a mandare i bambini a casa prima della fine delle lezioni.\n3. Dopo il titolo vinto, il presidente del club ha promesso di mandare i tifosi a casa felici e festanti.\n4. L'uragano ha mandato a casa gli oggetti lasciati incustoditi nel giardino volando attraverso il quartiere.\n5. Il meccanico, dopo aver riparato l'automobile, ha mandato il cliente a casa con il veicolo nuovamente funzionante.",
 "1. L'autista dell'autobus ha deciso di mandare a casa gli studenti prima del solito a causa della nevicata improvvisa.\n2. Il direttore, per premiare lo staff per il lavoro svolto, ha mandato a casa tutti i dipendenti due ore prima.\n3. A seguito dell'allarme incendio, hanno mandato a casa gli operai dal cantiere per motivi di sicurezza.\n4. La libreria online ha mandato a casa il pacco con i nuovi libri ordinati dal cliente.\n5.

In [689]:
structured_data = parse_and_structure_data(responses, idiom, category, type)

new_df= pd.DataFrame(structured_data, columns=["submission", "category", "idiom", "type"])
new_df

,submission,category,idiom,type
0,I vigili del fuoco hanno dovuto mandare tutti ...,letterale,mandare a casa,Enhanced-prompting
1,Il forte temporale ha spinto l'insegnante a ma...,letterale,mandare a casa,Enhanced-prompting
2,"Dopo il titolo vinto, il presidente del club h...",letterale,mandare a casa,Enhanced-prompting
3,L'uragano ha mandato a casa gli oggetti lascia...,letterale,mandare a casa,Enhanced-prompting
4,"Il meccanico, dopo aver riparato l'automobile,...",letterale,mandare a casa,Enhanced-prompting
5,L'autista dell'autobus ha deciso di mandare a ...,letterale,mandare a casa,Enhanced-prompting
6,"Il direttore, per premiare lo staff per il lav...",letterale,mandare a casa,Enhanced-prompting
7,"A seguito dell'allarme incendio, hanno mandato...",letterale,mandare a casa,Enhanced-prompting
8,La libreria online ha mandato a casa il pacco ...,letterale,mandare a casa,Enhanced-prompting
9,"Dopo il check-up, il veterinario ha mandato a ...",letterale,mandare a casa,Enhanced-prompting


In [690]:
new_df = new_df[new_df['submission']!=""]
new_df.reset_index(drop=True,inplace=True)
new_df


,submission,category,idiom,type
0,I vigili del fuoco hanno dovuto mandare tutti ...,letterale,mandare a casa,Enhanced-prompting
1,Il forte temporale ha spinto l'insegnante a ma...,letterale,mandare a casa,Enhanced-prompting
2,"Dopo il titolo vinto, il presidente del club h...",letterale,mandare a casa,Enhanced-prompting
3,L'uragano ha mandato a casa gli oggetti lascia...,letterale,mandare a casa,Enhanced-prompting
4,"Il meccanico, dopo aver riparato l'automobile,...",letterale,mandare a casa,Enhanced-prompting
5,L'autista dell'autobus ha deciso di mandare a ...,letterale,mandare a casa,Enhanced-prompting
6,"Il direttore, per premiare lo staff per il lav...",letterale,mandare a casa,Enhanced-prompting
7,"A seguito dell'allarme incendio, hanno mandato...",letterale,mandare a casa,Enhanced-prompting
8,La libreria online ha mandato a casa il pacco ...,letterale,mandare a casa,Enhanced-prompting
9,"Dopo il check-up, il veterinario ha mandato a ...",letterale,mandare a casa,Enhanced-prompting


In [691]:
df_structured = pd.concat([df_structured, new_df], ignore_index=True)
df_structured

,submission,category,idiom,type
0,"Manda queste scatole a casa subito, per favore.",letterale,mandare a casa,Zero-shot
1,"Ogni venerdì, il manager manda gli impiegati a...",letterale,mandare a casa,Zero-shot
2,"Durante l'allerta meteo, hanno mandato tutti g...",letterale,mandare a casa,Zero-shot
3,Potete mandare quel pacco grande al mio indiri...,letterale,mandare a casa,Zero-shot
4,Chi ha mandato queste lettere alla nostra casa?,letterale,mandare a casa,Zero-shot
...,...,...,...,...
95,"A causa della manifestazione in centro, l'azie...",letterale,mandare a casa,Enhanced-prompting
96,Il corriere ha suonato al campanello per manda...,letterale,mandare a casa,Enhanced-prompting
97,"Durante l'assemblea, il preside ha annunciato ...",letterale,mandare a casa,Enhanced-prompting
98,Il vento ha spazzato via i cappelli dei turist...,letterale,mandare a casa,Enhanced-prompting


In [692]:
df_structured

,submission,category,idiom,type
0,"Manda queste scatole a casa subito, per favore.",letterale,mandare a casa,Zero-shot
1,"Ogni venerdì, il manager manda gli impiegati a...",letterale,mandare a casa,Zero-shot
2,"Durante l'allerta meteo, hanno mandato tutti g...",letterale,mandare a casa,Zero-shot
3,Potete mandare quel pacco grande al mio indiri...,letterale,mandare a casa,Zero-shot
4,Chi ha mandato queste lettere alla nostra casa?,letterale,mandare a casa,Zero-shot
...,...,...,...,...
95,"A causa della manifestazione in centro, l'azie...",letterale,mandare a casa,Enhanced-prompting
96,Il corriere ha suonato al campanello per manda...,letterale,mandare a casa,Enhanced-prompting
97,"Durante l'assemblea, il preside ha annunciato ...",letterale,mandare a casa,Enhanced-prompting
98,Il vento ha spazzato via i cappelli dei turist...,letterale,mandare a casa,Enhanced-prompting


In [693]:
# Define the path for the CSV file
file_path = 'gpt4_it_gerçek.csv'

# Check if the file exists
if os.path.exists(file_path):
    # Append without writing headers
    df_structured.to_csv(file_path, mode='a', index=False, header=False)
else:
    # Create a new file and write the DataFrame with headers
    df_structured.to_csv(file_path, mode='w', index=False, header=True)